In [1]:
import lmql_demo.download as download

llama2_tokenizer = "llama2/llama-tokenizer.model"
mistral_tokenizer = "llama2/mistral-tokenizer.model"
model_filename = download.download_llama2()

The file 'llama2/llama-2-7b.Q4_K_M.gguf' already exists.


In [2]:
import lmql
from rich.pretty import pprint


In [3]:
local_model = lmql.model(f"local:llama.cpp:{model_filename}", tokenizer=llama2_tokenizer)

In [4]:
@lmql.query
async def sentiment_analysis(text: str) -> str:
    '''lmql
    """You are a sentiment analyzer. Given a text, you should return a sentiment label and
    a short one sentence explanation. Possible lables are POSITIVE, NEGATIVE and NEUTRAL.

    Example:
    text: I love this movie!
    thought: The word love indicates a positive sentiment.
    sentiment: POSITIVE

    text: The movie was quite dull.
    thought: The word dull conveys a negative sentiment.
    sentiment: NEGATIVE
    
    """
    "text: {text}\n"
    "thought: [THOUGHT]\n" where STOPS_BEFORE(THOUGHT,"\n") and len(TOKENS(THOUGHT)) < 100
    "sentiment:[SENTIMENT]" distribution SENTIMENT in [" POSITIVE", " NEGATIVE", " NEUTRAL"]
    '''

In [8]:
result = await(sentiment_analysis("Even though the hotel was a little spendy we loved our stay there.", model=local_model))
pprint(result)

LMQLResult(
│   prompt='You are a sentiment analyzer. Given a text, you should return a sentiment label and\na short one sentence explanation. Possible lables are POSITIVE, NEGATIVE and NEUTRAL.\nExample:\ntext: I love this movie!\nthought: The word love indicates a positive sentiment.\nsentiment: POSITIVE\ntext: The movie was quite dull.\nthought: The word dull conveys a negative sentiment.\nsentiment: NEGATIVE\ntext: Even though the hotel was a little spendy we loved our stay there.\nthought:  The word loved indicates a positive sentiment.\nsentiment: POSITIVE',
│   variables={
│   │   'THOUGHT': ' The word loved indicates a positive sentiment.',
│   │   'SENTIMENT': ' POSITIVE',
│   │   'P(SENTIMENT)': [
│   │   │   (' POSITIVE', 0.5604758702774101),
│   │   │   (' NEGATIVE', 0.1963316950580992),
│   │   │   (' NEUTRAL', 0.24319243466449064)
│   │   ],
│   │   'log P(SENTIMENT)': [
│   │   │   (' POSITIVE', -0.5789690877491398),
│   │   │   (' NEGATIVE', -1.6279497283895057),
│   │   │   (' NEUTRAL', -1.4139022368390601)
│   │   ]
│   },
│   distribution_variable='SENTIMENT',
│   distribution_values=[' POSITIVE', ' NEGATIVE', ' NEUTRAL'],
│   sequence=<detseq token_len=144 ids=[b'14088', b'263', b'6374', b'19688', b'29889', b'13', b'18616', b'2073', b'29901', b'2'] next_ids=[]>
)